In [13]:
import os
import json
import cv2
import random
import csv
from tqdm import tqdm

random.seed(42)

In [14]:
dataset_root = '/home/kvu/erc/meld/MELD-face-videos/'
dataset_json_path = dataset_root + 'datasets.json'
train = dataset_root + 'train'
dev = dataset_root + 'dev'
test = dataset_root + 'test'

output = 'single-frame-dataset'

In [15]:
with open(dataset_json_path) as f:
    dataset_json = json.load(f)

In [16]:
labels_mapping = {'anger': 0, 
                  'disgust': 1, 
                  'fear': 2, 
                  'joy': 3, 
                  'neutral': 4, 
                  'sadness': 5, 
                  'surprise': 6}

In [17]:
dataset_json['train']['dia0_utt0']

{'SrNo': '1',
 'Utterance': 'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
 'Speaker': 'Chandler',
 'Emotion': 'neutral',
 'Sentiment': 'neutral',
 'Dialogue_ID': '0',
 'Utterance_ID': '0',
 'Season': '8',
 'Episode': '21',
 'StartTime': '00:16:16,059',
 'EndTime': '00:16:21,731'}

In [18]:
len(os.listdir(train))

7523

In [23]:
def video2frame(set_name, source_video_folder, output_folder, output_csv_path):
    row_list = []
    row_list.append(['image_id', 'label'])

    list_dir = os.listdir(source_video_folder) 
    for vid in tqdm(list_dir):
        # Get video's metadata
        vid_name = vid[:-4]
        vid_capture = cv2.VideoCapture(source_video_folder + '/' + vid)
        vid_length = int(vid_capture.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
        emotion = dataset_json[set_name][vid_name]['Emotion']

        # Pick a random frame
        frame_index = random.randint(0, vid_length)
        vid_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

        ret, frame = vid_capture.read()
        output_path = output_folder + '/' + vid_name + '.jpg'
        cv2.imwrite(output_path, frame)

        image_id = vid_name + '.jpg'
        label = labels_mapping[emotion]
        row_list.append([image_id, label])

    with open(output_csv_path, 'w', newline='') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerows(row_list)

In [24]:
video2frame(set_name='train', 
            source_video_folder=train, 
            output_folder='/home/kvu/erc/single-frame-dataset/train', 
            output_csv_path='/home/kvu/erc/single-frame-dataset/train.csv')

100%|██████████| 7523/7523 [00:51<00:00, 146.30it/s]


In [25]:
video2frame(set_name='dev', 
            source_video_folder=dev, 
            output_folder='/home/kvu/erc/single-frame-dataset/dev', 
            output_csv_path='/home/kvu/erc/single-frame-dataset/dev.csv')

100%|██████████| 860/860 [00:06<00:00, 130.76it/s]


In [26]:
video2frame(set_name='test', 
            source_video_folder=test, 
            output_folder='/home/kvu/erc/single-frame-dataset/test', 
            output_csv_path='/home/kvu/erc/single-frame-dataset/test.csv')

100%|██████████| 1983/1983 [00:13<00:00, 148.90it/s]
